# 문제 1: 개와 고양이 분류 (Cats vs Dogs Classification)

## 과제 개요
- **데이터셋**: Kaggle Cats and Dogs (128×128 RGB, 2 클래스)
- **검증 방법**: 반복 홀드아웃 5회 (Repeated Holdout)
- **데이터 분할**: 훈련:검증 = 3:2 (60%:40%)
- **테스트**: 별도 테스트 데이터셋 사용
- **평가 척도**: Accuracy, F1 Score (Micro), F1 Score (Macro)

---

## 구현 모델 (5개)

| # | 모델 | 연도 | 핵심 특징 | 역할 |
|---|------|------|----------|------|
| 1 | **LeNet-5** | 1998 | 5×5 커널, Tanh, AvgPool | 베이스라인 (수업 모델) |
| 2 | **VGGNet** | 2014 | 3×3 커널 반복, BN+ReLU | 깊은 CNN |
| 3 | **ResNetCNN** | 2015 | Residual Connection | Skip Connection |
| 4 | **SEResNet** | 2017 | SE Block (채널 Attention) | Attention 메커니즘 |
| 5 | **ConvNeXt** | 2022 | Depthwise 7×7, LayerNorm, GELU | 최신 CNN 구조 |

---

## 평가 척도 설명

| 척도 | 계산 방법 | 특징 |
|------|----------|------|
| **Accuracy** | 정답 수 / 전체 수 | 전체 정확도 |
| **F1 (Micro)** | 전체 TP,FP,FN 합산 후 F1 계산 | 균형 데이터에서 = Accuracy |
| **F1 (Macro)** | 클래스별 F1의 산술 평균 | 클래스별 성능 균등 반영 |

---

## 실험 설계

```
[훈련 데이터] ──┬── 60% Train ──> 모델 학습
               └── 40% Val ────> Early Stopping 기준
                                  
[테스트 데이터] ──────────────────> 최종 성능 평가

위 과정을 시드를 바꿔가며 5회 반복 → 평균 ± 표준편차 보고
```

## 1. 라이브러리

In [ ]:
# -------------------------------
# 공용 라이브러리 및 유틸 불러오기
# - torch/torchvision: 모델 정의, 변환, Mixed Precision
# - sklearn: 데이터 분할 및 지표 계산
# - PIL: 이미지 로딩
# - matplotlib: 시각화 및 한글 폰트 설정
# - zipfile/os: Kaggle 데이터 압축 해제 및 경로 관리
# -------------------------------
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.amp import autocast, GradScaler  # Mixed Precision (새 API)
from torchvision import transforms
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import os
import zipfile

# 한글 폰트 설정 (Linux 서버 환경) - 그래프 라벨 깨짐 방지
def set_korean_font():
    """시스템에서 사용 가능한 한글 폰트 자동 설정"""
    font_candidates = [
        'NanumGothic', 'NanumBarunGothic', 'Malgun Gothic',
        'AppleGothic', 'DejaVu Sans', 'Noto Sans CJK KR'
    ]
    available_fonts = [f.name for f in fm.fontManager.ttflist]
    
    for font in font_candidates:
        if font in available_fonts:
            plt.rcParams['font.family'] = font
            plt.rcParams['axes.unicode_minus'] = False
            print(f"한글 폰트 설정: {font}")
            return
    
    # 폰트를 찾지 못한 경우 영문으로 대체
    print("한글 폰트를 찾지 못했습니다. 영문 라벨을 사용합니다.")

set_korean_font()

# GPU 우선, Mac이면 MPS, 그 외 CPU 선택
device = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')
print(f"Device: {device}")

# GPU 정보 출력 (리포트용)
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")



In [ ]:
# --------------------------------------------
# H100 GPU 최적화 기본 설정 + 시드 고정 헬퍼
# - 고정 배치/입력 크기를 가정하여 cuDNN benchmark 활성화
# - Mixed Precision 사용 여부는 USE_AMP로 통제
# --------------------------------------------
BATCH_SIZE = 128          # H100: 80GB 메모리, 배치 크기 증가
NUM_WORKERS = 8           # 데이터 로딩 병렬화
PIN_MEMORY = True         # GPU 메모리 전송 최적화
USE_AMP = True            # Mixed Precision (BF16/FP16)

def set_seed(seed):
    """torch/np 시드를 모두 고정하여 반복 홀드아웃 안정화"""
    torch.manual_seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)

# cuDNN 최적화 (고정 입력 크기에서 성능 향상)
torch.backends.cudnn.benchmark = True

print(f"H100 최적화 설정:")
print(f"  - Batch Size: {BATCH_SIZE}")
print(f"  - Num Workers: {NUM_WORKERS}")
print(f"  - Pin Memory: {PIN_MEMORY}")
print(f"  - Mixed Precision (AMP): {USE_AMP}")
print(f"  - cuDNN Benchmark: {torch.backends.cudnn.benchmark}")



## 2. 데이터셋 다운로드

Kaggle API 필요: `pip install kaggle`

kaggle.json을 ~/.kaggle/에 저장해야 함

In [ ]:
# --------------------------------------------
# Kaggle 데이터 다운로드를 위한 디렉토리 준비
# git에 데이터가 포함되지 않도록 ./data 하위에만 저장
# --------------------------------------------
DATA_DIR = './data/cats_dogs'
os.makedirs(DATA_DIR, exist_ok=True)



In [ ]:
# --------------------------------------------
# Kaggle 데이터셋 다운로드 (사전 준비: ~/.kaggle/kaggle.json)
# --------------------------------------------
!kaggle datasets download -d samuelcortinhas/cats-and-dogs-image-classification -p {DATA_DIR}



In [ ]:
# --------------------------------------------
# 다운로드한 zip 압축 해제
# train/test 디렉토리가 생성됨
# --------------------------------------------
zip_path = os.path.join(DATA_DIR, 'cats-and-dogs-image-classification.zip')
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(DATA_DIR)
print("압축 해제 완료")



## 3. 데이터 준비

In [ ]:
# --------------------------------------------
# Custom Dataset 정의
# - 폴더 구조: cats/, dogs/
# - 이미지 경로와 라벨을 미리 스캔하여 리스트로 보관
# - __getitem__에서 PIL 이미지를 로드 후 transform 적용
# --------------------------------------------
class CatsDogsDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.transform = transform
        self.image_paths = []
        self.labels = []
        
        for class_name, class_idx in [('cats', 0), ('dogs', 1)]:
            class_dir = os.path.join(data_dir, class_name)
            if os.path.exists(class_dir):
                for img_name in os.listdir(class_dir):
                    if img_name.endswith(('.jpg', '.jpeg', '.png')):
                        self.image_paths.append(os.path.join(class_dir, img_name))
                        self.labels.append(class_idx)
        
        # numpy 배열로 변환해 stratify 등에 사용
        self.labels = np.array(self.labels)
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        # PIL로 로드 후 RGB 고정, transform 적용
        image = Image.open(self.image_paths[idx]).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, self.labels[idx]



In [ ]:
# --------------------------------------------
# 데이터 변환 정의
# - train: 기본 리사이즈 후 수평 뒤집기/회전으로 데이터 증강
# - test: 검증/테스트는 증강 없이 정규화만 적용
# --------------------------------------------
train_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])



In [ ]:
# --------------------------------------------
# 데이터셋 로드 (train/test 분리 제공)
# --------------------------------------------
train_dir = os.path.join(DATA_DIR, 'train')
test_dir = os.path.join(DATA_DIR, 'test')

train_dataset = CatsDogsDataset(train_dir)
test_dataset = CatsDogsDataset(test_dir, transform=test_transform)

print(f"훈련: {len(train_dataset)}, 테스트: {len(test_dataset)}")
print(f"클래스 분포: cats={sum(train_dataset.labels==0)}, dogs={sum(train_dataset.labels==1)}")



In [ ]:
# --------------------------------------------
# 샘플 시각화
# - 무작위 위치 대신 일정 간격으로 샘플 선택 (i*50)
# - 정규화 해제 후 RGB로 표시
# --------------------------------------------
fig, axes = plt.subplots(2, 5, figsize=(12, 5))
temp_ds = CatsDogsDataset(train_dir, transform=test_transform)

for i, ax in enumerate(axes.flat):
    img, label = temp_ds[i * 50]
    # 정규화 해제: (x * std) + mean
    img = img * torch.tensor([0.229, 0.224, 0.225]).view(3,1,1) + torch.tensor([0.485, 0.456, 0.406]).view(3,1,1)
    ax.imshow(img.permute(1,2,0).clip(0,1))
    ax.set_title('Cat' if label == 0 else 'Dog')
    ax.axis('off')
plt.tight_layout()
plt.show()



## 4. 모델 정의

### 망 구조 표기법
- **C**: Convolution (합성곱) - 공간적 특징 추출
- **BN**: Batch Normalization - 학습 안정화, 수렴 가속
- **R**: ReLU / **T**: Tanh - 활성화 함수
- **AvgP** / **MaxP**: Average/Max Pooling - 다운샘플링
- **GAP**: Global Average Pooling - 파라미터 감소
- **D**: Dropout - 정규화, 과적합 방지
- **FC**: Fully Connected - 분류를 위한 선형 변환
- **Flat**: Flatten - 2D→1D 변환
- **Res**: Residual Connection - skip connection
- **SE**: Squeeze-and-Excitation - 채널별 attention

---

### 4.1 LeNet-5 (베이스라인, LeCun et al., 1998)

**구조**: `C(3,6,5×5) → T → AvgP → C(6,16,5×5) → T → AvgP → Flat → FC → T → FC → T → FC`

**특징**:
- 최초의 성공적인 CNN (수업시간에 배운 모델)
- 5×5 큰 커널로 특징 추출
- RGB 입력에 맞게 변형 (원본은 grayscale)

**효과/한계**:
- ✅ CNN의 기본 원리 적용
- ❌ 128×128 큰 이미지에는 얕은 구조
- ❌ 현대적 기법(BN, ReLU, Dropout) 미적용
- ❌ 개/고양이 같은 복잡한 분류에 한계

In [ ]:
class LeNet5(nn.Module):
    """
    LeNet-5 (LeCun et al., 1998) - 128x128 RGB 버전
    구조: C(3,6,5x5) → T → AvgP → C(6,16,5x5) → T → AvgP → C(16,32,5x5) → T → AvgP → Flat → FC → T → FC → T → FC
    
    - 원본 LeNet-5를 RGB 입력에 맞게 변형
    - 5x5 커널과 Tanh 활성화 (원본 논문)
    - 128x128 → 62x62 → 31x31 → 13x13 → 6x6 (AvgPool 추가)
    """
    def __init__(self, num_classes=2):
        super().__init__()
        # 망 구조를 ModuleList로 노출하여 각 레이어를 쉽게 확인
        self.layers = nn.ModuleList([
            # Feature Extraction (128x128 입력 처리)
            nn.Conv2d(3, 6, 5),                # C(3,6,5x5): 128 → 124
            nn.Tanh(),                          # T: 활성화
            nn.AvgPool2d(2, 2),                 # AvgP: 124 → 62
            nn.Conv2d(6, 16, 5),                # C(6,16,5x5): 62 → 58
            nn.Tanh(),                          # T: 활성화
            nn.AvgPool2d(2, 2),                 # AvgP: 58 → 29
            nn.Conv2d(16, 32, 5),               # C(16,32,5x5): 29 → 25
            nn.Tanh(),                          # T: 활성화
            nn.AvgPool2d(2, 2),                 # AvgP: 25 → 12
            nn.Flatten(),                       # Flat: (B,32,12,12) → (B,4608)
            # Classification
            nn.Linear(32 * 12 * 12, 120),       # FC(4608,120)
            nn.Tanh(),                          # T: 활성화
            nn.Linear(120, 84),                 # FC(120,84)
            nn.Tanh(),                          # T: 활성화
            nn.Linear(84, num_classes)          # FC(84,2): 출력층
        ])
    
    def forward(self, x):
        # ModuleList 순회로 순전파 진행
        for layer in self.layers:
            x = layer(x)
        return x

print("LeNet-5: C(5x5) → T → AvgP → C(5x5) → T → AvgP → C(5x5) → T → AvgP → Flat → FC → T → FC → T → FC")
print(LeNet5())
print(f"파라미터: {sum(p.numel() for p in LeNet5().parameters()):,}")



### 4.2 VGGNet-style (Simonyan & Zisserman, 2014)

**구조**: `[C(3×3) → BN → R]×2 → MaxP → [C(3×3) → BN → R]×2 → MaxP → [C(3×3) → BN → R]×3 → MaxP → [C(3×3) → BN → R]×3 → GAP → D → FC`

**특징**:
- 작은 3×3 커널을 깊게 쌓음 (VGG의 핵심 아이디어)
- 3×3 두 번 = 5×5 수용 영역, 더 적은 파라미터
- Batch Normalization + ReLU로 현대화
- MaxPooling으로 점진적 다운샘플링

**효과**:
- ✅ 깊은 네트워크로 복잡한 패턴 학습
- ✅ 작은 커널로 파라미터 효율성
- ✅ BN으로 학습 안정화
- ⚠️ Skip connection 없어 gradient 문제 가능

In [ ]:
class VGGBlock(nn.Module):
    """
    VGG Block: 3x3 Conv를 여러 번 쌓은 블록
    구조: [C(3x3) → BN → R] × n_convs
    """
    def __init__(self, in_ch, out_ch, n_convs=2):
        super().__init__()
        layers = []
        for i in range(n_convs):
            layers.extend([
                nn.Conv2d(in_ch if i == 0 else out_ch, out_ch, 3, 1, 1),  # C(3x3)
                nn.BatchNorm2d(out_ch),                                   # BN
                nn.ReLU()                                                  # R
            ])
        self.layers = nn.ModuleList(layers)
    
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x


class VGGNet(nn.Module):
    """
    VGGNet-style (Simonyan & Zisserman, 2014) - 128x128 RGB
    구조: VGGBlock×4 + MaxPool×4 + GAP + FC
    
    - 3x3 작은 커널만 사용 (VGG의 핵심)
    - BN과 ReLU로 현대화
    - 128x128 → 64 → 32 → 16 → 8 → GAP
    """
    def __init__(self, num_classes=2):
        super().__init__()
        # 망 구조: VGGBlock → MaxP → VGGBlock → MaxP → ... → GAP → D → FC
        self.layers = nn.ModuleList([
            # Block 1: 128x128 → 64x64
            VGGBlock(3, 32, n_convs=2),            # [C(3x3) → BN → R] × 2
            nn.MaxPool2d(2, 2),                     # MaxP
            # Block 2: 64x64 → 32x32
            VGGBlock(32, 64, n_convs=2),           # [C(3x3) → BN → R] × 2
            nn.MaxPool2d(2, 2),                     # MaxP
            # Block 3: 32x32 → 16x16
            VGGBlock(64, 128, n_convs=3),          # [C(3x3) → BN → R] × 3
            nn.MaxPool2d(2, 2),                     # MaxP
            # Block 4: 16x16 → 8x8
            VGGBlock(128, 256, n_convs=3),         # [C(3x3) → BN → R] × 3
            nn.MaxPool2d(2, 2),                     # MaxP: 8x8
            nn.AdaptiveAvgPool2d(1),               # GAP: 8x8 → 1x1
            nn.Dropout(0.5)                         # D: 과적합 방지
        ])
        self.fc = nn.Linear(256, num_classes)      # FC(256,2): 분류
    
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)

print("VGGNet: [C(3x3)→BN→R]×2 → MaxP → ... (4 blocks) → GAP → D → FC")
print(VGGNet())
print(f"파라미터: {sum(p.numel() for p in VGGNet().parameters()):,}")



### 4.3 ResNetCNN (He et al., 2015)

**구조**: `C(3,32,7×7,s2) → BN → R → MaxP → Res(32,64) → Res(64,128) → Res(128,256) → GAP → D(0.5) → FC(256,2)`

**ResBlock 내부**: `C(3×3) → BN → R → C(3×3) → BN → (+shortcut) → R`

**특징**:
- Residual Connection (skip connection) 도입
- 7×7 큰 커널로 시작 (넓은 수용 영역)
- MaxPool로 초기 다운샘플링
- 3개의 Residual Block

**효과**:
- ✅ Skip connection으로 gradient flow 개선
- ✅ 깊은 네트워크에서도 안정적인 학습
- ✅ VGGNet보다 적은 파라미터로 더 좋은 성능
- ✅ 개/고양이 같은 복잡한 분류에 효과적

In [ ]:
class ResidualBlock(nn.Module):
    """
    Residual Block (He et al., 2015)
    구조: C → BN → R → C → BN → (+shortcut) → R
    
    핵심 아이디어: H(x) = F(x) + x
    - 잔차 F(x)를 학습하여 gradient flow 개선
    """
    def __init__(self, in_ch, out_ch, stride=1):
        super().__init__()
        self.layers = nn.ModuleList([
            nn.Conv2d(in_ch, out_ch, 3, stride, 1, bias=False),  # C: 3x3 conv
            nn.BatchNorm2d(out_ch),                              # BN
            nn.ReLU(),                                           # R
            nn.Conv2d(out_ch, out_ch, 3, 1, 1, bias=False),      # C: 3x3 conv
            nn.BatchNorm2d(out_ch)                               # BN
        ])
        self.shortcut = nn.Sequential()
        if stride != 1 or in_ch != out_ch:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_ch, out_ch, 1, stride, bias=False),
                nn.BatchNorm2d(out_ch)
            )
    
    def forward(self, x):
        out = x
        for layer in self.layers:
            out = layer(out)
        out += self.shortcut(x)  # Residual: F(x) + x
        return nn.ReLU()(out)


class ResNetCNN(nn.Module):
    """
    ResNetCNN (He et al., 2015) - 128x128 RGB
    구조: C(7x7,s2) → BN → R → MaxP → Res(32,64) → Res(64,128) → Res(128,256) → GAP → D → FC
    
    - 7x7 큰 커널로 시작 (넓은 수용 영역)
    - 3개의 Residual Block
    """
    def __init__(self, num_classes=2):
        super().__init__()
        self.layers = nn.ModuleList([
            nn.Conv2d(3, 32, 7, 2, 3, bias=False),   # C(3,32,7x7): 128 → 64
            nn.BatchNorm2d(32),                      # BN
            nn.ReLU(),                               # R
            nn.MaxPool2d(3, 2, 1),                   # MaxP: 64 → 32
            ResidualBlock(32, 64, 1),                # Res: 32x32
            ResidualBlock(64, 128, 2),               # Res: 16x16
            ResidualBlock(128, 256, 2),              # Res: 8x8
            nn.AdaptiveAvgPool2d(1),                 # GAP: 1x1
            nn.Dropout(0.5)                          # D
        ])
        self.fc = nn.Linear(256, num_classes)
    
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)

print("ResNetCNN: C(7x7) → BN → R → MaxP → Res → Res → Res → GAP → D → FC")
print(ResNetCNN())
print(f"파라미터: {sum(p.numel() for p in ResNetCNN().parameters()):,}")



### 4.4 SEResNet (Hu et al., 2017) - 최고 성능 예상

**구조**: `C(3,32,7×7,s2) → BN → R → MaxP → SERes(32,64) → SERes(64,128) → SERes(128,256) → GAP → D(0.5) → FC(256,2)`

**SEResBlock 내부**: `C → BN → R → C → BN → SE → (+shortcut) → R`

**SE Block**: `GAP → FC(C,C/16) → R → FC(C/16,C) → Sigmoid → Scale`
- Squeeze: GAP로 채널별 전역 정보 압축
- Excitation: FC로 채널 간 관계 학습 후 중요도 계산

**특징**:
- Squeeze-and-Excitation 메커니즘
- 채널별 attention으로 털 패턴, 귀 모양 등 중요 특징 강조
- 수업에서 다루지 않은 최신 구조

**효과**:
- ✅ 채널별 중요도를 동적으로 조절
- ✅ 개/고양이 구분에 중요한 특징 강조
- ✅ 적은 파라미터 추가로 큰 성능 향상
- ✅ Residual + Attention 시너지

In [ ]:
class SEBlock(nn.Module):
    """
    Squeeze-and-Excitation Block (Hu et al., 2017)
    구조: GAP → FC(C, C/r) → R → FC(C/r, C) → Sigmoid → Scale
    """
    def __init__(self, channels, reduction=16):
        super().__init__()
        self.layers = nn.ModuleList([
            nn.AdaptiveAvgPool2d(1),                          # Squeeze
            nn.Flatten(),
            nn.Linear(channels, channels // reduction),       # FC: 축소
            nn.ReLU(),
            nn.Linear(channels // reduction, channels),       # FC: 복원
            nn.Sigmoid()
        ])
    
    def forward(self, x):
        b, c, _, _ = x.size()
        scale = x
        for layer in self.layers:
            scale = layer(scale)
        return x * scale.view(b, c, 1, 1)


class SEResidualBlock(nn.Module):
    """
    SE-Residual Block
    구조: C → BN → R → C → BN → SE → (+shortcut) → R
    """
    def __init__(self, in_ch, out_ch, stride=1):
        super().__init__()
        self.layers = nn.ModuleList([
            nn.Conv2d(in_ch, out_ch, 3, stride, 1, bias=False),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(),
            nn.Conv2d(out_ch, out_ch, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_ch),
            SEBlock(out_ch)
        ])
        self.shortcut = nn.Sequential()
        if stride != 1 or in_ch != out_ch:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_ch, out_ch, 1, stride, bias=False),
                nn.BatchNorm2d(out_ch)
            )
    
    def forward(self, x):
        out = x
        for layer in self.layers:
            out = layer(out)
        out += self.shortcut(x)
        return nn.ReLU()(out)


class SEResNet(nn.Module):
    """
    SE-ResNet (Hu et al., 2017) - 128x128 RGB
    구조: C(7x7) → BN → R → MaxP → SERes×3 → GAP → D → FC
    """
    def __init__(self, num_classes=2):
        super().__init__()
        self.layers = nn.ModuleList([
            nn.Conv2d(3, 32, 7, 2, 3, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(3, 2, 1),
            SEResidualBlock(32, 64, 1),
            SEResidualBlock(64, 128, 2),
            SEResidualBlock(128, 256, 2),
            nn.AdaptiveAvgPool2d(1),
            nn.Dropout(0.5)
        ])
        self.fc = nn.Linear(256, num_classes)
    
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)

print("SEResNet: C(7x7) → BN → R → MaxP → SERes → SERes → SERes → GAP → D → FC")
print(SEResNet())
print(f"파라미터: {sum(p.numel() for p in SEResNet().parameters()):,}")



### 4.5 ConvNeXt-Tiny (Liu et al., 2022) - 최신 CNN 구조

**배경**: Vision Transformer(ViT)의 성공 이후, CNN도 Transformer의 설계 원칙을 도입하여 성능을 끌어올릴 수 있음을 보여준 모델

**구조**: `Stem(4×4,s4) → [ConvNeXtBlock × {3,3,9,3}] → GAP → D → FC`

**ConvNeXtBlock 내부**:
```
DWConv(7×7) → LayerNorm → PWConv(×4) → GELU → PWConv → LayerScale → (+Residual)
```

**핵심 설계 원칙 (ViT에서 차용)**:
1. **Depthwise Conv 7×7**: Transformer의 큰 수용영역 모방 (ViT: 16×16 패치)
2. **LayerNorm**: BatchNorm 대신 사용 (Transformer 스타일)
3. **GELU 활성화**: ReLU 대신 GELU (Transformer와 동일)
4. **Inverted Bottleneck**: 채널을 4배 확장 후 축소 (MobileNet/ViT MLP)
5. **LayerScale**: 학습 안정화를 위한 학습 가능한 스케일 파라미터
6. **Patchify Stem**: 4×4 stride 4 conv로 초기 다운샘플링 (ViT 패치 임베딩과 유사)

**효과**:
- ✅ ViT 수준의 성능을 CNN으로 달성
- ✅ Transformer 대비 학습 효율성 (더 적은 데이터/연산)
- ✅ 수업에서 다루지 않은 2022년 최신 구조
- ✅ ImageNet에서 ResNet, ViT를 능가하는 성능

In [ ]:
# --------------------------------------------
# ConvNeXt-Tiny 스타일 고성능 CNN (128x128 RGB)
# - Depthwise 7x7 → LayerNorm → Pointwise 확장/축소 (GELU)
# - Stage별 다운샘플을 포함한 4-stage ConvNeXt-Tiny 변형
# --------------------------------------------
class LayerNorm2d(nn.Module):
    """채널 차원 기준 LayerNorm (ConvNeXt에서 사용)"""
    def __init__(self, num_channels, eps=1e-6):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(num_channels))
        self.bias = nn.Parameter(torch.zeros(num_channels))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(dim=1, keepdim=True)
        var = x.var(dim=1, keepdim=True, unbiased=False)
        x = (x - mean) / torch.sqrt(var + self.eps)
        return self.weight.view(1, -1, 1, 1) * x + self.bias.view(1, -1, 1, 1)


class ConvNeXtBlock(nn.Module):
    """
    ConvNeXt Block
    구조: DWConv(7x7) → LN → PWConv(×4) → GELU → PWConv → LayerScale → Residual
    """
    def __init__(self, dim, layer_scale_init_value=1e-6):
        super().__init__()
        self.dwconv = nn.Conv2d(dim, dim, kernel_size=7, padding=3, groups=dim)
        self.norm = LayerNorm2d(dim)
        self.pwconv1 = nn.Conv2d(dim, 4 * dim, kernel_size=1)
        self.act = nn.GELU()
        self.pwconv2 = nn.Conv2d(4 * dim, dim, kernel_size=1)
        self.gamma = nn.Parameter(layer_scale_init_value * torch.ones(dim)) if layer_scale_init_value > 0 else None

    def forward(self, x):
        shortcut = x
        x = self.dwconv(x)
        x = self.norm(x)
        x = self.pwconv1(x)
        x = self.act(x)
        x = self.pwconv2(x)
        if self.gamma is not None:
            x = self.gamma.view(1, -1, 1, 1) * x
        return shortcut + x


class ConvNeXtTiny(nn.Module):
    """
    ConvNeXt-Tiny 변형 (이미지넷 대비 축소, 128x128 입력)
    구조: Stem(4x4,s4) → [Block×{3,3,9,3} + Downsample] → GAP → Dropout → FC
    """
    def __init__(self, num_classes=2):
        super().__init__()
        depths = [3, 3, 9, 3]
        dims = [96, 192, 384, 768]

        # Stage별 다운샘플 레이어 (Stem 포함)
        self.downsample_layers = nn.ModuleList([
            nn.Sequential(
                nn.Conv2d(3, dims[0], kernel_size=4, stride=4),  # 128 → 32
                LayerNorm2d(dims[0])
            )
        ])
        for i in range(3):
            self.downsample_layers.append(
                nn.Sequential(
                    LayerNorm2d(dims[i]),
                    nn.Conv2d(dims[i], dims[i+1], kernel_size=2, stride=2)  # 해상도 절반
                )
            )

        # ConvNeXt Blocks
        self.stages = nn.ModuleList()
        for i in range(4):
            blocks = [ConvNeXtBlock(dims[i]) for _ in range(depths[i])]
            self.stages.append(nn.ModuleList(blocks))

        self.norm = LayerNorm2d(dims[-1])
        self.head = nn.Linear(dims[-1], num_classes)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = self.downsample_layers[0](x)
        for i in range(4):
            for block in self.stages[i]:
                x = block(x)
            if i < 3:
                x = self.downsample_layers[i+1](x)
        x = self.norm(x)
        x = x.mean([-2, -1])  # GAP
        x = self.dropout(x)
        return self.head(x)

print("ConvNeXt-Tiny: Stem(4x4,s4) → Blocks×{3,3,9,3} → GAP → D → FC")
print(ConvNeXtTiny())
print(f"파라미터: {sum(p.numel() for p in ConvNeXtTiny().parameters()):,}")



## 5. 학습 함수

In [ ]:
# --------------------------------------------
# 학습/평가 유틸리티 함수 (H100 최적화)
# - GradScaler로 underflow 방지
# - autocast로 BF16/FP16 연산
# - scheduler/early stopping은 train_model에서 관리
# --------------------------------------------
# Mixed Precision을 위한 GradScaler (H100 최적화)
scaler = GradScaler('cuda', enabled=USE_AMP)

def train_epoch(model, loader, criterion, optimizer):
    """Mixed Precision 학습 (H100 최적화)"""
    model.train()
    for images, labels in loader:
        images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)
        
        optimizer.zero_grad(set_to_none=True)
        
        # AMP: forward/backward를 자동 캐스팅
        with autocast('cuda', enabled=USE_AMP):
            outputs = model(images)
            loss = criterion(outputs, labels)
        
        # 스케일된 그래디언트로 업데이트
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()


def evaluate(model, loader):
    """평가 함수 - Accuracy, F1 (Micro), F1 (Macro) 반환"""
    model.eval()
    preds, labels_list = [], []
    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device, non_blocking=True)
            with autocast('cuda', enabled=USE_AMP):
                outputs = model(images)
            preds.extend(outputs.argmax(1).cpu().numpy())
            labels_list.extend(labels.numpy())
    
    acc = accuracy_score(labels_list, preds)
    f1_micro = f1_score(labels_list, preds, average='micro')
    f1_macro = f1_score(labels_list, preds, average='macro')
    return acc, f1_micro, f1_macro

def train_model(model, train_loader, val_loader, epochs=20, lr=0.001):
    """H100 최적화 학습 함수"""
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=0.01)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
    
    best_acc = 0
    best_state = None
    patience = 0
    
    for epoch in range(epochs):
        train_epoch(model, train_loader, criterion, optimizer)
        val_acc, _, _ = evaluate(model, val_loader)
        scheduler.step()
        
        if val_acc > best_acc:
            best_acc = val_acc
            best_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
            patience = 0
        else:
            patience += 1
            if patience >= 7:
                break
    
    # 가장 성능 좋은 스냅샷 복원
    if best_state:
        model.load_state_dict(best_state)
    return model

print("H100 최적화 학습 함수 정의 완료")
print("  - F1 Score: Micro (=Accuracy) + Macro (클래스별 평균)")



## 6. 반복 홀드아웃 검증 (5회)

In [ ]:
# --------------------------------------------
# 인덱스 기반 부분집합을 transform과 함께 래핑하는 헬퍼
# --------------------------------------------
class TransformDataset(Dataset):
    def __init__(self, base_dataset, indices, transform):
        self.base = base_dataset
        self.indices = indices
        self.transform = transform
    
    def __len__(self):
        return len(self.indices)
    
    def __getitem__(self, idx):
        i = self.indices[idx]
        image = Image.open(self.base.image_paths[i]).convert('RGB')
        return self.transform(image), self.base.labels[i]



In [ ]:
# --------------------------------------------
# 반복 홀드아웃 실행 함수
# - Stratified split으로 3:2(train:val) 분할을 5회 반복
# - torch.compile로 추가 최적화 시도 (가능한 경우)
# - 테스트 세트는 고정
# --------------------------------------------
def run_holdout(model_class, model_name, n_repeats=5):
    """H100 최적화 반복 홀드아웃"""
    results = {'accuracy': [], 'f1_micro': [], 'f1_macro': []}
    all_indices = np.arange(len(train_dataset))
    
    print()  # 구분을 위한 빈 줄
    print('='*60)
    print(model_name)
    print('='*60)
    
    test_loader = DataLoader(
        test_dataset, 
        batch_size=BATCH_SIZE,
        num_workers=NUM_WORKERS,
        pin_memory=PIN_MEMORY
    )
    
    for i in range(n_repeats):
        set_seed(42 + i)
        
        # 3:2 분할 (train:val)
        train_idx, val_idx = train_test_split(
            all_indices, test_size=0.4, stratify=train_dataset.labels, random_state=42+i
        )
        
        train_subset = TransformDataset(train_dataset, train_idx, train_transform)
        val_subset = TransformDataset(train_dataset, val_idx, test_transform)
        
        train_loader = DataLoader(
            train_subset, 
            batch_size=BATCH_SIZE, 
            shuffle=True,
            num_workers=NUM_WORKERS,
            pin_memory=PIN_MEMORY,
            drop_last=True
        )
        val_loader = DataLoader(
            val_subset, 
            batch_size=BATCH_SIZE,
            num_workers=NUM_WORKERS,
            pin_memory=PIN_MEMORY
        )
        
        model = model_class().to(device)
        
        # PyTorch 2.x: torch.compile로 추가 최적화 시도
        if hasattr(torch, 'compile'):
            try:
                model = torch.compile(model, mode='reduce-overhead')
            except:
                pass
        
        model = train_model(model, train_loader, val_loader, epochs=30)
        
        acc, f1_micro, f1_macro = evaluate(model, test_loader)
        results['accuracy'].append(acc)
        results['f1_micro'].append(f1_micro)
        results['f1_macro'].append(f1_macro)
        print(f"[{i+1}/{n_repeats}] Acc: {acc:.4f}, F1(Micro): {f1_micro:.4f}, F1(Macro): {f1_macro:.4f}")
        
        del model
        torch.cuda.empty_cache() if torch.cuda.is_available() else None
    
    return results



In [ ]:
lenet_results = run_holdout(LeNet5, "LeNet-5 (베이스라인)")

In [ ]:
vgg_results = run_holdout(VGGNet, "VGGNet-style")

In [ ]:
resnet_results = run_holdout(ResNetCNN, "ResNetCNN")

In [ ]:
senet_results = run_holdout(SEResNet, "SEResNet (최고 성능)")

In [ ]:
convnext_results = run_holdout(ConvNeXtTiny, "ConvNeXt-Tiny (고성능)")


## 7. 결과 분석

In [ ]:
def print_summary(results, name):
    # 평균/표준편차를 빠르게 확인하기 위한 헬퍼
    acc_mean, acc_std = np.mean(results['accuracy']), np.std(results['accuracy'])
    f1_micro_mean, f1_micro_std = np.mean(results['f1_micro']), np.std(results['f1_micro'])
    f1_macro_mean, f1_macro_std = np.mean(results['f1_macro']), np.std(results['f1_macro'])
    print(f"{name}")
    print(f"  Accuracy:    {acc_mean:.4f} ± {acc_std:.4f}")
    print(f"  F1 (Micro):  {f1_micro_mean:.4f} ± {f1_micro_std:.4f}")
    print(f"  F1 (Macro):  {f1_macro_mean:.4f} ± {f1_macro_std:.4f}")
    return acc_mean, acc_std, f1_micro_mean, f1_micro_std, f1_macro_mean, f1_macro_std

print("="*60)
print("최종 결과 (Final Results)")
print("="*60)
lenet_stats = print_summary(lenet_results, "LeNet-5")
print()
vgg_stats = print_summary(vgg_results, "VGGNet")
print()
resnet_stats = print_summary(resnet_results, "ResNetCNN")
print()
senet_stats = print_summary(senet_results, "SEResNet")
print()
convnext_stats = print_summary(convnext_results, "ConvNeXt-Tiny")



In [ ]:
# --------------------------------------------
# 시각화 - 모든 평가 지표 (Accuracy, F1 Micro, F1 Macro) 비교
# - 한글 폰트 재설정 (그래프 출력 전 확인)
# - 3개 지표를 하나의 그래프에 그룹으로 표시
# --------------------------------------------

# 한글 폰트 재설정 (시각화 전 확인)
import platform
if platform.system() == 'Darwin':  # macOS
    plt.rcParams['font.family'] = 'AppleGothic'
elif platform.system() == 'Windows':
    plt.rcParams['font.family'] = 'Malgun Gothic'
else:  # Linux
    # 사용 가능한 한글 폰트 찾기
    font_list = [f.name for f in fm.fontManager.ttflist if 'Nanum' in f.name or 'Gothic' in f.name]
    if font_list:
        plt.rcParams['font.family'] = font_list[0]
plt.rcParams['axes.unicode_minus'] = False

# 모델별 결과 정리
models = ['LeNet-5', 'VGGNet', 'ResNetCNN', 'SEResNet', 'ConvNeXt']
all_results = [lenet_results, vgg_results, resnet_results, senet_results, convnext_results]
colors = ['#ff9999', '#66b3ff', '#99ff99', '#ffcc99', '#c2a3ff']

# ============================================
# 그래프 1: 모든 평가 지표 비교 (평균 ± 표준편차)
# ============================================
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# 1. Accuracy
acc_means = [np.mean(r['accuracy']) for r in all_results]
acc_stds = [np.std(r['accuracy']) for r in all_results]
bars1 = axes[0].bar(models, acc_means, yerr=acc_stds, capsize=5, color=colors, edgecolor='black', linewidth=0.5)
axes[0].set_title('Accuracy (정확도)', fontsize=12, fontweight='bold')
axes[0].set_ylim([0.5, 1.0])
axes[0].set_ylabel('Score')
axes[0].tick_params(axis='x', rotation=15)
# 막대 위에 수치 표시
for bar, mean, std in zip(bars1, acc_means, acc_stds):
    axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + std + 0.01, 
                 f'{mean:.3f}', ha='center', va='bottom', fontsize=9)

# 2. F1 Score (Micro)
f1_micro_means = [np.mean(r['f1_micro']) for r in all_results]
f1_micro_stds = [np.std(r['f1_micro']) for r in all_results]
bars2 = axes[1].bar(models, f1_micro_means, yerr=f1_micro_stds, capsize=5, color=colors, edgecolor='black', linewidth=0.5)
axes[1].set_title('F1 Score (Micro)', fontsize=12, fontweight='bold')
axes[1].set_ylim([0.5, 1.0])
axes[1].set_ylabel('Score')
axes[1].tick_params(axis='x', rotation=15)
for bar, mean, std in zip(bars2, f1_micro_means, f1_micro_stds):
    axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + std + 0.01, 
                 f'{mean:.3f}', ha='center', va='bottom', fontsize=9)

# 3. F1 Score (Macro)
f1_macro_means = [np.mean(r['f1_macro']) for r in all_results]
f1_macro_stds = [np.std(r['f1_macro']) for r in all_results]
bars3 = axes[2].bar(models, f1_macro_means, yerr=f1_macro_stds, capsize=5, color=colors, edgecolor='black', linewidth=0.5)
axes[2].set_title('F1 Score (Macro)', fontsize=12, fontweight='bold')
axes[2].set_ylim([0.5, 1.0])
axes[2].set_ylabel('Score')
axes[2].tick_params(axis='x', rotation=15)
for bar, mean, std in zip(bars3, f1_macro_means, f1_macro_stds):
    axes[2].text(bar.get_x() + bar.get_width()/2, bar.get_height() + std + 0.01, 
                 f'{mean:.3f}', ha='center', va='bottom', fontsize=9)

plt.suptitle('Cats vs Dogs - 모델별 성능 비교 (5회 반복 홀드아웃)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('task1_metrics_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

# ============================================
# 그래프 2: 세 지표를 하나의 그래프에 그룹으로 표시
# ============================================
fig, ax = plt.subplots(figsize=(14, 6))

x = np.arange(len(models))
width = 0.25

# 세 지표를 나란히 배치
bars_acc = ax.bar(x - width, acc_means, width, yerr=acc_stds, label='Accuracy', color='#4ECDC4', capsize=3, edgecolor='black', linewidth=0.5)
bars_micro = ax.bar(x, f1_micro_means, width, yerr=f1_micro_stds, label='F1 (Micro)', color='#FF6B6B', capsize=3, edgecolor='black', linewidth=0.5)
bars_macro = ax.bar(x + width, f1_macro_means, width, yerr=f1_macro_stds, label='F1 (Macro)', color='#45B7D1', capsize=3, edgecolor='black', linewidth=0.5)

ax.set_xlabel('모델', fontsize=12)
ax.set_ylabel('Score', fontsize=12)
ax.set_title('Cats vs Dogs - 평가 지표별 모델 성능 비교', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(models, fontsize=10)
ax.legend(loc='lower right', fontsize=10)
ax.set_ylim([0.5, 1.0])
ax.grid(axis='y', alpha=0.3)

# 수치 표시 (Accuracy만)
for bar, mean in zip(bars_acc, acc_means):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02, 
            f'{mean:.3f}', ha='center', va='bottom', fontsize=8, rotation=0)

plt.tight_layout()
plt.savefig('task1_grouped_metrics.png', dpi=150, bbox_inches='tight')
plt.show()

# ============================================
# 그래프 3: 반복별 성능 추이 (F1 Macro)
# ============================================
fig, ax = plt.subplots(figsize=(12, 5))
x = np.arange(5)
width = 0.15

for i, (result, model, color) in enumerate(zip(all_results, models, colors)):
    offset = (i - 2) * width
    ax.bar(x + offset, result['f1_macro'], width, label=model, color=color, edgecolor='black', linewidth=0.5)

ax.set_xlabel('반복 횟수 (Iteration)', fontsize=12)
ax.set_ylabel('F1 Score (Macro)', fontsize=12)
ax.set_title('Cats vs Dogs - 반복별 F1 (Macro) 성능', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels([f'{i+1}회' for i in x], fontsize=10)
ax.legend(loc='lower right', fontsize=9)
ax.set_ylim([0.5, 1.0])
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('task1_iteration_f1macro.png', dpi=150, bbox_inches='tight')
plt.show()

# ============================================
# 결과 요약 테이블 출력
# ============================================
print("\n" + "="*80)
print("📊 Cats vs Dogs - 최종 결과 요약 (5회 반복 홀드아웃)")
print("="*80)
print(f"{'모델':<15} {'Accuracy':<20} {'F1 (Micro)':<20} {'F1 (Macro)':<20}")
print("-"*80)
for model, result in zip(models, all_results):
    acc_m, acc_s = np.mean(result['accuracy']), np.std(result['accuracy'])
    f1mi_m, f1mi_s = np.mean(result['f1_micro']), np.std(result['f1_micro'])
    f1ma_m, f1ma_s = np.mean(result['f1_macro']), np.std(result['f1_macro'])
    print(f"{model:<15} {acc_m:.4f} ± {acc_s:.4f}      {f1mi_m:.4f} ± {f1mi_s:.4f}      {f1ma_m:.4f} ± {f1ma_s:.4f}")
print("="*80)
print("\n※ 참고: 균형 데이터셋에서 F1 (Micro) = Accuracy 입니다.")
print("※ F1 (Macro)는 각 클래스(고양이/개)의 F1을 평균하여 클래스별 성능을 균등하게 반영합니다.")


## 8. 결론

### 모델별 성능 비교

| 모델 | 연도 | 핵심 구조 | 파라미터 | 예상 성능 |
|------|------|----------|---------|----------|
| LeNet-5 | 1998 | C(5×5)-Tanh-AvgP | ~560K | 낮음 (베이스라인) |
| VGGNet | 2014 | [C(3×3)-BN-R]×n-MaxP | ~1.9M | 중간 |
| ResNetCNN | 2015 | Residual Block (Skip) | ~600K | 높음 |
| SEResNet | 2017 | SE Block + Residual | ~620K | 높음 |
| ConvNeXt | 2022 | DWConv(7×7)-LN-GELU | ~28M | 최고 |

※ 실제 성능은 위 시각화 결과 참조

---

### 평가 척도 해석

| 척도 | 계산 방식 | 이 데이터셋에서의 의미 |
|------|----------|---------------------|
| **Accuracy** | (TP+TN) / 전체 | 전체 정확도 |
| **F1 (Micro)** | 전체 TP,FP,FN 합산 | 균형 데이터 → Accuracy와 동일 |
| **F1 (Macro)** | 클래스별 F1 평균 | 고양이/개 각각의 성능 균등 반영 |

※ 균형 데이터셋에서 F1 (Micro) = Accuracy는 sklearn의 정상 동작입니다.

---

### 모델별 상세 분석

#### 1. LeNet-5 (베이스라인)
- **구조**: 5×5 큰 커널 + Tanh 활성화 + AvgPool
- **특징**: 1998년 고전적 CNN, 수업시간에 배운 모델
- **한계**: 얕은 구조(3 conv)로 128×128 이미지의 복잡한 패턴 학습에 한계
- **예상 성능**: 가장 낮음

#### 2. VGGNet-style
- **구조**: 3×3 작은 커널을 깊게 쌓음 (10 conv)
- **특징**: 3×3 두 번 = 5×5 수용영역, 더 적은 파라미터
- **개선점**: BN + ReLU로 현대화
- **한계**: Skip connection 없어 깊어질수록 gradient vanishing 가능

#### 3. ResNetCNN
- **구조**: Residual Block (H(x) = F(x) + x)
- **특징**: Skip connection으로 gradient flow 개선
- **효과**: VGGNet보다 적은 파라미터로 더 깊은 학습 가능
- **예상 성능**: VGGNet보다 높음

#### 4. SEResNet
- **구조**: ResNet + SE Block (채널 Attention)
- **SE Block**: GAP → FC → ReLU → FC → Sigmoid → Scale
- **효과**: 채널별 중요도 동적 조절 (털 패턴, 귀 모양 등 강조)
- **예상 성능**: ResNet보다 높음

#### 5. ConvNeXt-Tiny (2022 최신)
- **구조**: ViT 설계 원칙을 CNN에 적용
- **핵심 요소**:
  - Depthwise 7×7 conv (큰 수용영역)
  - LayerNorm (BatchNorm 대체)
  - GELU 활성화 (ReLU 대체)
  - Inverted Bottleneck (채널 4배 확장 후 축소)
- **특징**: ImageNet에서 ViT, ResNet을 능가하는 성능
- **예상 성능**: 가장 높음 (파라미터 수 대비 성능 최고)

---

### CNN 발전 흐름 요약

```
LeNet-5 (1998)     → 기초적 CNN 구조 확립
       ↓
VGGNet (2014)      → 작은 커널을 깊게 쌓는 방식
       ↓
ResNet (2015)      → Skip Connection으로 깊은 학습 가능
       ↓
SENet (2017)       → Channel Attention 도입
       ↓
ConvNeXt (2022)    → Transformer 설계 원칙을 CNN에 적용
```

### 개 vs 고양이 분류에서 중요한 특징

| 특징 | 개 | 고양이 | 구분 난이도 |
|------|-----|--------|------------|
| 귀 형태 | 늘어진/뾰족한 | 삼각형 | 중간 |
| 코 형태 | 긴 주둥이 | 짧은 주둥이 | 중간 |
| 눈 형태 | 다양 | 세로 동공 | 쉬움 |
| 털 패턴 | 다양 | 줄무늬/단색 | 어려움 |
| 전체 형태 | 다양한 크기 | 비교적 균일 | 중간 |

→ **SE Attention**이 털 패턴, 귀 모양 등 중요한 특징 채널을 강조하는 데 유리

이 프로젝트에서는 CNN의 발전 역사를 따라가며 각 기법의 효과를 실험적으로 비교합니다.